In [6]:
import paramiko
import numpy as np
import itertools
import os

# ===================== SSH CONFIG =====================
SSH_HOST = "10.173.154.77"                
SSH_PORT = 22                           
SSH_USERNAME = "pburton"             
SSH_PASSWORD = "pbucf564"               
REMOTE_FILE_PATH = "/mnt/disk_ssd1/home/pburton/product/RuNMe2OH2O.out" #file path here
LOCAL_FILE_PATH = "Test1.txt"  #store locally

def download_file_via_ssh():
    try:
        print("Connecting to SSH client...")
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
        ssh.connect(SSH_HOST, port=SSH_PORT, username=SSH_USERNAME, password=SSH_PASSWORD)
        sftp = ssh.open_sftp()
        print(f"Downloading {REMOTE_FILE_PATH}...")
        sftp.get(REMOTE_FILE_PATH, LOCAL_FILE_PATH)  
        sftp.close()
        ssh.close()
        print("Download complete. Saved as", LOCAL_FILE_PATH)
    except Exception as e:
        print("connection error", e)

# ===================== ATOMS =====================
atomic_number_to_symbol = {
    1: "H",    2: "He",  3: "Li",  4: "Be",  5: "B",
    6: "C",    7: "N",   8: "O",   9: "F",   10: "Ne",
    11: "Na",  12: "Mg", 13: "Al", 14: "Si", 15: "P",
    16: "S",   17: "Cl", 18: "Ar", 19: "K",  20: "Ca",
    21: "Sc",  22: "Ti", 23: "V",  24: "Cr", 25: "Mn",
    26: "Fe",  27: "Co", 28: "Ni", 29: "Cu", 30: "Zn",
    31: "Ga",  32: "Ge", 33: "As", 34: "Se", 35: "Br",
    36: "Kr",  37: "Rb", 38: "Sr", 39: "Y",  40: "Zr",
    41: "Nb",  42: "Mo", 43: "Tc", 44: "Ru", 45: "Rh",
    46: "Pd",  47: "Ag", 48: "Cd", 49: "In", 50: "Sn",
    51: "Sb",  52: "Te", 53: "I",  54: "Xe", 55: "Cs",
    56: "Ba",  57: "La", 58: "Ce", 59: "Pr", 60: "Nd",
    61: "Pm",  62: "Sm", 63: "Eu", 64: "Gd", 65: "Tb",
    66: "Dy",  67: "Ho", 68: "Er", 69: "Tm", 70: "Yb",
    71: "Lu",  72: "Hf", 73: "Ta", 74: "W",  75: "Re",
    76: "Os",  77: "Ir", 78: "Pt", 79: "Au", 80: "Hg",
    81: "Tl",  82: "Pb", 83: "Bi", 84: "Po", 85: "At",
    86: "Rn",  87: "Fr", 88: "Ra", 89: "Ac", 90: "Th",
    91: "Pa",  92: "U",  93: "Np", 94: "Pu", 95: "Am"
}

def element_from_atomic_number(num):
    return atomic_number_to_symbol.get(num, str(num))

# ===================== GAUSSIAN COORDINATES =====================
def read_gaussian_coordinates(file_path):

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    block_start = None
    for i, line in enumerate(lines):
        if "Input orientation:" in line:
            block_start = i

    if block_start is None:
        print("No 'Input orientation:' block found in", file_path)
        return []

    coord_start = block_start + 5 

    coordinates = []
    for line in lines[coord_start:]:
        stripped = line.strip()
        if stripped.startswith("-----"):
            break
        parts = stripped.split()
        if len(parts) >= 6:
            try:
                atomic_num = int(parts[1])
                x = float(parts[3])
                y = float(parts[4])
                z = float(parts[5])
                symbol = element_from_atomic_number(atomic_num)
                coordinates.append((symbol, x, y, z))
            except ValueError:
                continue
    return coordinates

# ===================== Metal Charge =====================
def extract_central_metal_charge(file_path):

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    charge_block_start = None
    for i, line in enumerate(lines):
        if "Mulliken" in line:
            charge_block_start = i
            break

    if charge_block_start is None:
        print("No 'Mulliken atomic charges:' block found in", file_path)
        return None

    charges = []
    for line in lines[charge_block_start+2:]:
        stripped = line.strip()
        if stripped == "" or "-------" in stripped:
            break
        parts = stripped.split()
        if len(parts) >= 2:
            try:
                charge_value = float(parts[1])
                charges.append(charge_value)
            except ValueError:
                continue

    atomic_data = read_gaussian_coordinates(file_path)
    metal_atoms = {"Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
                   "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",
                   "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg"}
    for idx, (symbol, x, y, z) in enumerate(atomic_data):
        if symbol in metal_atoms:
            if idx < len(charges):
                print(f"Central metal atom: {symbol} with Mulliken charge: {charges[idx]} in {file_path}")
                return charges[idx]
    print("No metal :(", file_path)
    return None

# ===================== Bond Length From Coordinates=====================
def calculate_bond_lengths(atomic_data, threshold=2.0):
    bonds = []
    for (i, atom1), (j, atom2) in itertools.combinations(enumerate(atomic_data), 2):
        symbol1, x1, y1, z1 = atom1
        symbol2, x2, y2, z2 = atom2
        bond_length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
        if bond_length <= threshold:
            bonds.append((symbol1, symbol2, bond_length))
    return bonds

# ===================== Process Path (Needs work)*** =====================
def process_input_path(input_path):
    """
    Processes the input path, which can be either a single file or a folder.
    If it's a folder, the function sequentially reads all .out files in that folder
    and prints the data for each complex.
    """
    if os.path.isdir(input_path):
        print("\nProcessing folder:", input_path)
        # Debug: print the list of files in the folder
        folder_files = os.listdir(input_path)
        print("Files found in folder:", folder_files)
        
        out_files = sorted(f for f in folder_files if f.lower().endswith(".out"))
        if not out_files:
            print("No '.out' files found in the folder.")
        for filename in out_files:
            file_path = os.path.join(input_path, filename)
           
            complex_name = os.path.splitext(filename)[0]
            print("\nComplex:", complex_name)
            atomic_data = read_gaussian_coordinates(file_path)
            metal_charge = extract_central_metal_charge(file_path)
            bonds = calculate_bond_lengths(atomic_data, threshold=2.0)
            print("Central Metal Charge:", metal_charge)
            if bonds:
                print("Atomic Bonds (within threshold):")
                print("{:<5} {:<5} {:<15}".format("Atom1", "Atom2", "Bond Length (Å)"))
                print("-" * 30)
                for a1, a2, length in bonds:
                    print(f"{a1:<5} {a2:<5} {length:<15.4f}")
            else:
                print("No bonds found.")
            print("-" * 40)
    elif os.path.isfile(input_path):
        print("\nProcessing file:", input_path)
        
        if os.path.abspath(input_path) == os.path.abspath(LOCAL_FILE_PATH):
            complex_name = os.path.splitext(os.path.basename(REMOTE_FILE_PATH))[0]
        else:
            complex_name = os.path.splitext(os.path.basename(input_path))[0]
        print("\nComplex:", complex_name)
        atomic_data = read_gaussian_coordinates(input_path)
        metal_charge = extract_central_metal_charge(input_path)
        bonds = calculate_bond_lengths(atomic_data, threshold=2.0)
        print("Central Metal Charge:", metal_charge)
        if bonds:
            print("Atomic Bonds (within threshold):")
            print("{:<5} {:<5} {:<15}".format("Atom1", "Atom2", "Bond Length (Å)"))
            print("-" * 30)
            for a1, a2, length in bonds:
                print(f"{a1:<5} {a2:<5} {length:<15.4f}")
        else:
            print("No bonds found.")
    else:
        print(f"The provided path '{input_path}' is neither a valid file nor a folder.")

# ===================== Execution**** =====================
if __name__ == "__main__":
    download_file_via_ssh() 

    input_path = LOCAL_FILE_PATH  
    process_input_path(input_path)



Connecting to SSH client...
Download complete. Saved as Test1.txt

Processing file: Test1.txt

Complex: RuNMe2OH2O
Central metal atom: Ru with Mulliken charge: 2.0 in Test1.txt
Central Metal Charge: 2.0
Atomic Bonds (within threshold):
Atom1 Atom2 Bond Length (Å)
------------------------------
O     H     0.9796         
H     O     0.9840         
N     C     1.4604         
N     C     1.4617         
N     C     1.4589         
N     C     1.4585         
N     C     1.4598         
N     C     1.4637         
N     C     1.4635         
N     C     1.4629         
C     H     1.0878         
C     H     1.1009         
C     H     1.0997         
H     H     1.7825         
H     H     1.7855         
H     H     1.7725         
C     H     1.1016         
C     H     1.0992         
C     H     1.0898         
H     H     1.7765         
H     H     1.7922         
H     H     1.7950         
C     H     1.0877         
C     H     1.1017         
C     H     1.1012         
H    